In [1]:
import numpy as np

# ENCODING

In [2]:
def getTextToTransmit():
    with open("input.txt", "r") as f:
        text_to_transmit = f.read()
    return text_to_transmit;

def encodeText( text ):
    #each character is a vector of 8 bits
    return np.unpackbits(bytearray(text,"utf-8"))

def convoluteEncoding(bits):
    b=[1,1]+list(map(lambda x:(x-0.5)*2,bits))+[1,1]
    x=[xi for j in range(2,len(b)) for xi in [b[j]*b[j-2],b[j]*b[j-1]*b[j-2]]]
    return x

def scaleArrayKTimes(k,bits):
    return [b for b in bits for i in range(k)]

def addCalibrationZeros(k,bits):
    return np.concatenate((np.zeros(k),bits))

def writeChannelInput(bits):
    with open("encoded.txt", "w") as f:
        stringToSend = "\n".join(map(lambda bit:str(int(bit)),bits))
        f.write(stringToSend)

In [3]:
MAX_SIZE=51200
NB_BYTES=80
BITS_PER_BYTES=8
ENCODER_FACTOR=2
NB_FINAL_DUMMY_BITS=2
BITS_ARRAY_SIZE=(NB_BYTES*BITS_PER_BYTES+NB_FINAL_DUMMY_BITS)*ENCODER_FACTOR
print("BITS_ARRAY_SIZE :",BITS_ARRAY_SIZE)

REPEAT_FACTOR= MAX_SIZE//(BITS_ARRAY_SIZE)-1
print("REPEAT_FACTOR :",REPEAT_FACTOR)

CALIBRATION_SIZE=BITS_ARRAY_SIZE
print('BITS_ARRAY_SIZE * REPEAT_FACTOR + CALIBRATION_SIZE =',BITS_ARRAY_SIZE,'*',REPEAT_FACTOR,'+',CALIBRATION_SIZE,'=',BITS_ARRAY_SIZE*REPEAT_FACTOR+CALIBRATION_SIZE)

x=addCalibrationZeros(CALIBRATION_SIZE,scaleArrayKTimes(REPEAT_FACTOR,convoluteEncoding(encodeText(getTextToTransmit()))))


print("input length :",len(x),"maximum is",MAX_SIZE)

BITS_ARRAY_SIZE : 1284
REPEAT_FACTOR : 38
BITS_ARRAY_SIZE * REPEAT_FACTOR + CALIBRATION_SIZE = 1284 * 38 + 1284 = 50076
input length : 50076 maximum is 51200


In [4]:
writeChannelInput(x)

# DECODING

In [5]:
def getChannelOutput():
    with open("received.txt", "r") as f:
        bitArray = np.array(f.read().split('\n'))
        return list(map(lambda x: float(x),bitArray[:-1]))
    #return list(x)
    
def getBitsFromY(Y):
    bias=np.mean(Y[:CALIBRATION_SIZE])
    print("bias =",bias)
    
    #group by bloc of size 38 starting at 1280 -> get 1280 bits
    bits=[np.mean(Y[CALIBRATION_SIZE+j*REPEAT_FACTOR:CALIBRATION_SIZE+(j+1)*REPEAT_FACTOR])-bias for j in range(BITS_ARRAY_SIZE)]
    
    return bits

def dot(a,b):
    return a[0]*b[0]+a[1]*b[1]

In [6]:
y=getBitsFromY(getChannelOutput())

bias = 0.056394401915164226


In [7]:
print("Nombre de bits =",len(y))
print("Il y a",int(len(y)/2),"pairs")

Nombre de bits = 1284
Il y a 642 pairs


In [8]:
nb_states=int(len(y)/2)
#attention on ajoute un etat de plus pour
#compter l'état 0
states_scores=np.zeros((nb_states+1,4))

previous_was=np.zeros((nb_states,4),dtype='int')

In [9]:
#To prevent from taking those path on the fist state
states_scores[0][1]=float("-inf")
states_scores[0][2]=float("-inf")
states_scores[0][3]=float("-inf")

#To prevent from taking those path on the second state
states_scores[1][2]=float("-inf")
states_scores[1][3]=float("-inf")

In [10]:
print(states_scores.shape)
print(previous_was.shape)

(643, 4)
(642, 4)


In [11]:
trellis=[]
trellis.append(((0,(1,1)),(2,(-1,-1))))
trellis.append(((0,(-1,-1)),(2,(1,1))))
trellis.append(((1,(1,-1)),(3,(-1,1))))
trellis.append(((1,(-1,1)),(3,(1,-1))))

for t in trellis:
    print(t)

((0, (1, 1)), (2, (-1, -1)))
((0, (-1, -1)), (2, (1, 1)))
((1, (1, -1)), (3, (-1, 1)))
((1, (-1, 1)), (3, (1, -1)))


In [12]:
bit_from_previous=[1,-1,1,-1]

In [13]:
for i in range(1,len(states_scores)):
    for j in range(4):
        a=states_scores[i-1][trellis[j][0][0]]+dot(trellis[j][0][1],y[(i-1)*2:i*2])
        b=states_scores[i-1][trellis[j][1][0]]+dot(trellis[j][1][1],y[(i-1)*2:i*2])
        
        states_scores[i][j]=max(a,b)
        previous_was[i-1][j]= trellis[j][0][0] if a>b else trellis[j][1][0]

In [14]:
j=0
bits=[]
for i in range(len(previous_was))[::-1]:
    prev=previous_was[i][j]
    
    bits=[bit_from_previous[j]]+bits
    
    j=prev


In [15]:
bits=list(map(lambda x:int((x+1)/2),bits))

In [16]:
binaryResult = np.array(bits[:-2])

binary_decoded = np.packbits(binaryResult.reshape(-1, 8)) 

decoded = []
for i in range(0, len(binary_decoded)):
    char = binary_decoded[i].tostring().decode("utf-8")
    decoded.append(char)

with open("output.txt", "w") as f:
    f.write("".join(decoded))